In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
reasons_df.head()

,call_id,primary_call_reason
0,call_id,primary_call_reason
1,4667960400,Voluntary Cancel
2,1122072124,Booking
3,6834291559,IRROPS
4,2266439882,Upgrade


In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

reasons_df = pd.read_csv('/content/reason18315ff.csv', header=None, names=['call_id', 'primary_call_reason'])

calls_df = pd.read_csv('/content/callsf0d4f5a.csv')

test_df = pd.read_csv('/content/testbc7185d.csv')

calls_df['call_id'] = calls_df['call_id'].astype(str)  # Convert to string
reasons_df['call_id'] = reasons_df['call_id'].astype(str)  # Convert to string

In [12]:
reasons_df.head()

,call_id,primary_call_reason
0,call_id,primary_call_reason
1,4667960400,Voluntary Cancel
2,1122072124,Booking
3,6834291559,IRROPS
4,2266439882,Upgrade


In [14]:
merged_df = pd.merge(calls_df, reasons_df, on='call_id', how='left')
merged_df['primary_call_reason'] = merged_df['primary_call_reason'].fillna("")  # Replace NaNs with empty strings

def preprocess_text(text):
    filler_words = ['uhh', 'umm', 'yeah', 'okay', 'alright', 'like', 'you know', 'agent', 'customer', 'know',
                    'really', 'rest', 'high', 'help', 'let', 'look', 'pull', 'hi', 'thanks', 'youre', 'typing']
    text = ' '.join(word for word in text.split() if word.lower() not in filler_words)

    agent_phrases = [
        r'thank you for calling united airlines customer service.*?',
        r'thank you for your patience.*?',
        r'(my name is .*? how may i help you\?)',
        r'(thank you for flying with us.*)'
    ]
    for phrase in agent_phrases:
        text = re.sub(phrase, '', text, flags=re.IGNORECASE)

    contractions = {"don't": "do not", "can't": "cannot", "i'm": "i am", "it's": "it is", "isn't": "is not"}
    for contraction, full_form in contractions.items():
        text = re.sub(contraction, full_form, text, flags=re.IGNORECASE)

    text = re.sub(r'[^a-zA-Z\s\?\!]', '', text)
    text = text.lower()

    domain_specific_stopwords = ['flight', 'airline', 'reservation', 'book', 'united']
    all_stopwords = stop_words.union(domain_specific_stopwords)
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in all_stopwords)

    return text

merged_df['cleaned_transcript'] = merged_df['call_transcript'].apply(preprocess_text)


In [18]:
merged_df['primary_call_reason'] = merged_df['primary_call_reason'].fillna("")

merged_df = merged_df[(merged_df['cleaned_transcript'] != "") & (merged_df['primary_call_reason'] != "")]

texts = merged_df['cleaned_transcript']
labels = merged_df['primary_call_reason']

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
labels_encoded = label_tokenizer.texts_to_sequences(labels)
labels_encoded = [item[0] - 1 for item in labels_encoded if item]

if len(texts) != len(labels_encoded):
    print(f"Inconsistent lengths: texts: {len(texts)}, labels_encoded: {len(labels_encoded)}")
    texts = texts[labels_encoded != []]
    labels_encoded = [item for item in labels_encoded if item]

X_train, X_test, y_train, y_test = train_test_split(texts, labels_encoded, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max(len(x) for x in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_tokenizer.word_index) + 1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
import numpy as np
y_train = np.array(y_train)
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.2)

predictions = model.predict(X_test_pad)
predicted_classes = predictions.argmax(axis=1)

results_df = pd.DataFrame({'call_id': X_test.index,
                           'predicted_reason': [label_tokenizer.index_word[i + 1] for i in predicted_classes]})

output_file_path = 'predictions_deep_learning.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")

Epoch 1/10
1202/1202 [==============================] - 193s 159ms/step - loss: 2.5397 - accuracy: 0.1923 - val_loss: 2.4956 - val_accuracy: 0.1842
Epoch 2/10
1202/1202 [==============================] - 191s 159ms/step - loss: 2.5037 - accuracy: 0.2012 - val_loss: 2.4922 - val_accuracy: 0.2089
Epoch 3/10
1202/1202 [==============================] - 192s 159ms/step - loss: 2.4913 - accuracy: 0.2035 - val_loss: 2.4819 - val_accuracy: 0.2071
Epoch 4/10
1202/1202 [==============================] - 188s 156ms/step - loss: 2.4777 - accuracy: 0.2134 - val_loss: 2.4740 - val_accuracy: 0.2118
Epoch 5/10
1202/1202 [==============================] - 193s 160ms/step - loss: 2.4625 - accuracy: 0.2167 - val_loss: 2.4697 - val_accuracy: 0.2159
Epoch 6/10
1202/1202 [==============================] - 191s 159ms/step - loss: 2.4441 - accuracy: 0.2262 - val_loss: 2.4598 - val_accuracy: 0.2183
Epoch 7/10
1202/1202 [==============================] - 188s 157ms/step - loss: 2.4172 - accuracy: 0.2364 - val_